In [1]:
import os

os.chdir('examples')

# Natural language processing

We have learned how to work with data organized as an [array](arrays.ipynb) of numbers, using the `numpy` package, or more generally as a [table](data_analysis.ipynb) of numeric and text columns, using `pandas`. We have also learned a bit about [working with image files](images.ipynb). The other major data format that you are likely to encounter in many computer programming tasks is natural, human-language text.

In computing, people often refer to the languages that human beings use for everyday communication, such as English or Pirahá, as 'natural' languages. This distinguishes them from programming languages like Python, and helps avoid confusion, for example when asking a computer person 'What language do you use at work?' Using computers to analyze natural language is often called 'Natural Language Processing' (NLP).

Unlike humans, computers still have huge difficulty responding to natural language appropriately. Here we will introduce a few tools and techniques that can help if we need to write a computer program that takes natural language text as its input. These techniques are necessary, often as an initial step, in many common programming tasks:

* translating between natural languages
* classifying texts by topic
* identifying particular tones or styles, for example rude or offensive posts in a forum
* creating chatbots or generating automated responses to customer queries
* ... and even some genuinely useful things like [establishing the authorship of anonymous works of literature](https://dl.acm.org/doi/10.5555/1314498.1314541)

Real-world natural language processing tasks like these are usually quite complex, and require a 'pipeline' of multiple techniques. As usual, we will only cover the basics. After our short introduction, you should know at least where to start if you later come to work with natural language text data.

## Objective

Here is our toy task for this lesson.

Let's imagine we have landed a job as an editor at a pretentious elitist newspaper, and we would like to write a program that helps us identify all the sentences in a text that violate one of the woefully misguided rules of the newspaper's style handbook. Let's take three example rules:

* A preposition (such as for, to, etc.) is not a good thing to end a sentence with.
* To boldly place an intervening word between the `to` and the `verb` in an infinitive is forbidden.
* And you shouldn't begin a sentence with a conjunction (such as and, but, etc.)

This isn't a completely unrealistic example. Talking right is something that many people still get [very excited](https://www.theguardian.com/books/booksblog/2015/oct/06/steven-pinker-alleged-rules-of-writing-superstitions) about.

So that we have a large example text to work with, let's load the classic go-to text for all NLP examples ever, *Moby Dick*:

In [2]:
import os

md = open(os.path.join('data', 'melville-moby_dick.txt')).read()

print(md[:433])

[Moby Dick by Herman Melville 1851]


ETYMOLOGY.

(Supplied by a Late Consumptive Usher to a Grammar School)

The pale Usher--threadbare in coat, heart, body, and brain; I see him
now.  He was ever dusting his old lexicons and grammars, with a queer
handkerchief, mockingly embellished with all the gay flags of all the
known nations of the world.  He loved to dust his old grammars; it
somehow mildly reminded him of his mortality.



## String methods again

Let's dive right in and just take a naive stab at the first rule, about not having a preposition at the end of a sentence. Ignoring for a moment any initial processing, such as splitting up the full text into sentences, we can first write a [function](glossary.ipynb#function) to determine whether or not a single sentence violates the rule.

Remember the [main ingredients of a function](functions.ipynb#Defining-functions):

* **the name of the function**. This function's job is just to say 'yes' or 'no'. For such functions, a good choice of name is some abbreviated form of a 'yes/no' question. We can go with `startswith_prep`.

* **its input [arguments](glossary.ipynb#argument)**. This is easy. The single input argument is a [string](glossary.ipynb) containing the sentence.

* **what steps it carries out (the 'body' of the function)**. There is room for some variation and choice of strategy here. But a simple start would be to split the sentence into words, get the final one, remove any punctuation characters, then compare it to a list of prepositions.

* **the [return value](glossary.ipynb#return)**. We don't actually want to return the printed answer `'yes'` or `'no'`. If our function is to be useful as part of a bigger program, it needs to return the computer versions of 'yes' and 'no', a [boolean](glossary.ipynb#boolean), with `True` for 'yes the sentence violates the rule' and `False` for 'no it does not'.

In [3]:
punctuation = ' .,?!:;'
prepositions = ['around', 'at', 'by', 'down', 'from', 'in', 'of', 'on', 'out', 'to', 'up', 'with']

def endswith_preposition(sentence):
    words = sentence.split()
    final_word = words[-1].strip(punctuation)
    return final_word.lower() in prepositions

Let's give our function a quick test.

In [4]:
endswith_preposition("This is the sort of mindless pedantry I just can't put up with!")

True

In [5]:
endswith_preposition("This is the sort of mindless pedantry up with which I just can't put!")

False

It works. But you can probably already spot a few deficiencies. Most obviously, our pre-defined list of prepositions is far from complete.

In [6]:
endswith_preposition('What are you talking about?')

False

We can fix this sort of omission by including more prepositions. But there are [a lot of prepositions](https://en.wikipedia.org/wiki/List_of_English_prepositions) to take into account.

There is a more subtle problem, too. Some words have different meanings in different contexts, and might or might not be prepositions depending on how they are used. So our function may produce some false positives.

In [7]:
endswith_preposition('Beneath their tougher outer feathers, birds have an insulating layer of down.')

True

The problem of context when processing natural language is particularly tricky. Computers don't understand the meanings of words and phrases. Getting a computer to correctly categorize different senses of the same word or phrase across varying contexts is an open problem, and one for which there are currently only partial and fragile solutions available, even at the cutting edge of NLP. In part this may be because it is a genuinely very difficult problem. Humans don't always get it right either.

*How do you get down from an elephant?*

*You don't, you get down from a duck.*

## nltk

As we have seen a few times before, when we encounter what seems like a difficult but common problem, there may be some tools already out there that can help save us from re-inventing the wheel. There are lots of Python [packages](glossary.ipynb#package) for NLP, which apply various techniques ranging from simple string processing like in our example function above, to powerful but complicated machine learning.

We will look at just one NLP package, called `nltk`. It is a but more complex than our example function, but still simple enough to be suitable for an introduction to NLP. Indeed, `nltk` was created specifically in order to teach linguists how to do computing with Python, and it accompanies a university class and a book about NLP, which is [free to read online](http://www.nltk.org/book). `nltk` is included in the default Anaconda installation, so if you have Anaconda then you will have it already.